# cool-title-here

**Group Members:** Aaron Go, John Alonzo, Sean Olores, Sean Cardeno 

**Deadline:** March 21, 2026 (Saturday), 8:00 AM

**Task:** Predict **the Challenge Rating of a monster** based on **the monster's features**

**Dataset Source:** [DnD 5e Monsters](https://www.kaggle.com/datasets/mrpantherson/dnd-5e-monsters/data)

**Justification:** In Dungeons & Dragons, *Challenge Rating* is a mathematical estimation of a monster's combat efficiency. A model that learns to predict this value from raw stats allows Game Masters to instantly balance custom *homebrew* content without manually calculating complex design formulas.


## Section 1: Data Preparation

In this section, we will:
- Load the dataset
- Inspect the data structure and basic statistics
- Handle missing values and data cleaning
- Identify data types and any inconsistencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('dnd_monsters.csv') 

In [ ]:
# Data Cleaning
# TODO: Do the missing stuff that other notebooks fixed like the aldani in row 22 
# TODO: Convert fractional CR strings (like "1/4") into decimals (0.25)
# TODO: Ordinal Encode the 'Size' column (Tiny=0, Small=1, etc.) per tips and tricks slides
# do basic cleaning stuff JIC

## Section 2: Exploratory Data Analysis (EDA)

In this section, we will:
- Analyze the distribution of features
- Identify relationships between features and the target variable
- Generate visualizations to understand the data better
- Compute statistical summaries

In [ ]:
# Exploratory Data Analysis
# TODO
# Dont forget to export visualizations for poster

## Section 3: Feature Engineering & Preprocessing

In this section, we will:
- Separate features and target variable
- Encode categorical variables
- Split data into training, validation, and test sets
- Normalize/standardize features if needed
- Perform feature selection if necessary

In [ ]:
# TODO: Define target variable and features
# TODO: Parse 'Speed' column into has_fly and has_swim

# TODO: Extract sub-race from 'Type' column (like extracting Elf from Humanoid)

# TODO: Standardize/Scale the numeric features for the Neural Network

## Section 4: Model Selection and Training

In this section, we will:
- Select and train **Classical Model 1** 
- Select and train **Classical Model 2** 
- Select and train **Neural Network Model** 
- Provide justification for model choices
- Train models and monitor performance on validation set

### Classical Model 1: 
Justification: 

### Classical Model 2: 
Justification: 

### Neural Network Model: 
Justification: 

## Section 5: Error Analysis and Model Tuning

In this section, we will:
- Analyze errors made by the models
- Identify patterns in prediction errors [i forgor were doing regression]
- Perform hyperparameter tuning
- Improve model performance through adjustments
- Select the best model based on validation performance

In [ ]:
# Error Analysis
# TODO

## Section 6: Model Evaluation

In this section, we will:
- Evaluate all models on the test set
- Compare performance metrics across models
- Generate detailed evaluation reports
- Visualize performance comparisons
- Identify the best performing model

## Section 7: Conclusions and Findings


### **Summary of Results:**
- 

### **Best Model Details:**
- 

### **Future Improvements:**
- 

### **References:**
- 

### **AI Disclosure:**
- 